# Building blocks for programming preconditioners

In [ ]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
%gui tk
from ngsolve.la import EigenValues_Preconditioner
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

In [ ]:
fes = H1(mesh, order=3, dirichlet="left|bottom")
u, v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
a.Assemble()
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()
gfu = GridFunction(fes)

## Create a Jacobi-preconditioner

Let  $A=$ `a.mat` be the assembled matrix, which can be decomposed based on `FreeDofs` ($F$) the remainder ($D$), as in $\S$[1.3](unit-1.3-dirichlet/dirichlet.ipynb),

$$
A = \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right). 
$$

Then the matrix form of the **point Jacobi preconditioner** is

$$
J = \left( \begin{array}{cc} \text{diag}(A_{FF})^{-1} & 0  \\ 0 & I  \end{array} \right),
$$

which can be obtained in NGSolve using `CreateSmoother`:

In [ ]:
preJpoint = a.mat.CreateSmoother(fes.FreeDofs())

NGSolve also gives us a facility to quickly check an estimate of the condition number of the preconditioned matrix by applying the Lanczos algorithm on the preconditioned system.


In [ ]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preJpoint)
lams

An estimate of the condition number 
$$
\kappa = \frac{\lambda_{\text{max}} }{ \lambda_{\text{min}}}
$$
is therefore given as follows:

In [ ]:
max(lams)/min(lams)

One might wonder if we have gained anything by this point Jacobi preconditioning. What if we did not precondition at all?

Not preconditioning is the same as preconditioning by the identity operator on $F$-dofs. One way to realize this identity operator in NGSolve is through the projection into the space of free dofs (i.e., the space spanned by the shape functions corresponding to free dofs). NGSolve provides

```
Projector(mask, range)   # mask: bit array, range: bool
```

which projects into the space spanned by the shape functions of the degrees of freedom marked as range in mask.

In [ ]:
preI = Projector(mask=fes.FreeDofs(), range=True)

*Note*  that another way to obtain the identity matrix in NGSolve is    
```
IdentityMatrix(fes.ndof, complex=False).
```

In [ ]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
max(lams)/min(lams)

Clearly the point Jacobi preconditioner has reduced the condition number. 

We can use preconditioners within iterative solvers provided by NGSolve's `solvers` module (which has `MinRes`, `QMR` etc.) Here is an illustration of its use within CG, or the **conjugate gradient** solver: 

In [ ]:
solvers.CG(mat=a.mat, pre=preJpoint, rhs=f.vec, sol=gfu.vec)
Draw(gfu)

##  Gauss-Seidel smoothing

The *same* point Jacobi smoother object can also used to perform **point Gauss-Seidel** smoothing. One step of the classical Gauss-Seidel iteration is realized by the method `preJpoint.Smooth()`. Its well known that this iteration converges for matrices like $A$. Below we show how to use it as a linear iterative solver. 


In [ ]:
gfu.vec[:] = 0
res = f.vec.CreateVector()              # residual 
projres = f.vec.CreateVector()          # residual projected to freedofs
proj = Projector(fes.FreeDofs(), True)

for i in range(500):
    preJpoint.Smooth(gfu.vec, f.vec)    # one step of point Gauss-Seidel
    res.data = f.vec - a.mat*gfu.vec      
    projres.data = proj * res
    print ("it#", i, ", res =", Norm(projres))
Draw (gfu)

## Implement a forward-backward GS preconditioner

The *same* point Jacobi smoother object is also able to perform a Gauss-Seidel iteration after reversing the ordering of the points, i.e., a **backward** Gauss-Seidel sweep. One can combine the forward and backward sweeps to construct a symmetric preconditioner, often called the **symmetrized Gauss-Seidel preconditioner**. This offers a good illustration of how to construct NGSolve preconditioners from within python. 

In [ ]:
class SymmetricGS(BaseMatrix):
    def __init__ (self, smoother):
        super(SymmetricGS, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y,x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [ ]:
preGS = SymmetricGS(preJpoint)
solvers.CG(mat=a.mat, pre=preGS, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

In [ ]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preGS)
max(lams)/min(lams)

Note that the condition number now is better than that of the system preconditioned by point Jacobi.

## A Block Jacobi preconditioner

The point Jacobi preconditioner is based on inverses of 1 x 1 diagonal blocks.  Condition numbers can be improved by using larger blocks. It is possible to group dofs into blocks within python and construct an NGSolve preconditioner based on the blocks.

Here is an example that constructs vertex-based blocks.

In [ ]:
blocks = []
freedofs = fes.FreeDofs()
for v in mesh.vertices:
    vdofs = set()
    for el in mesh[v].elements:
        vdofs |= set(d for d in fes.GetDofNrs(el) if freedofs[d])
    blocks.append (vdofs)
print (blocks)

`CreateBlockSmoother` can now take these blocks and construct a block Jacobi preconditioner.

In [ ]:
blockjac = a.mat.CreateBlockSmoother(blocks)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockjac)
max(lams)/min(lams)

Multiplicative smoothers and its symmetrized version often yield better condition numbers in practice. We can apply the same code we wrote above for symmetrization (`SymmetricGS`) to the block smoother:

In [ ]:
blockgs = SymmetricGS(blockjac)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockgs)
max(lams)/min(lams)

## Add a coarse grid correction

Dependence of the condition number on degrees of freedom can often be reduced by preconditioners that appropriately use a coarse grid correction.  It is also possible to experiment with coarse grid corrections using NGSolve's python interface. We now show how to precondition with a coarse grid correction made using the lowest order subspace of `fes`.

In the example below, note that we use `fes.GetDofNrs` again. Previously we used it with argument `el` of type `ElementId`, while now we use it with an argument `v` of type `MeshNode`.

In [ ]:
vertexdofs = BitArray(fes.ndof)
vertexdofs[:] = False

for v in mesh.vertices:
    for d in fes.GetDofNrs(v):
        vertexdofs[d] = True
        
vertexdofs &= fes.FreeDofs()

print(vertexdofs)    # bit array, printed 50 chars/line

Thus we have made a mask `vertexdofs` which reveals all free dofs associated to vertices. If these are labeled $c$ (and the remainder is labeled $f$), then the matrix $A$ can partitioned into 
$$
A = \left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right). 
$$
The matrix `coarsepre` below represents
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right). 
$$

In [ ]:
coarsepre = a.mat.Inverse(vertexdofs)

This matrix can be used for coarse grid correction. 

*Pitfall!*  Note that `coarsepre` is not appropriate as a preconditioner by itself as it has a large null space. You might get the wrong idea from the results of a Lanczos eigenvalue estimation:

In [ ]:
EigenValues_Preconditioner(mat=a.mat, pre=coarsepre)

But this result only gives the Laczos eigenvalue estimates on the *range* of the preconditioner. The preconditioned operator in this case is simply 
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right)
\left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right)
 = 
 \left( \begin{array}{cc} I  & A_{cc}^{-1} A_{cf} \\ 0 & 0  \end{array} \right),
$$
which is a projection into the $c$-dofs. Hence its no surprise that Lanczos estimated the eigenvalues of this operator (on its range) to be just one. But this does not imply that the condition number of this preconditioned system is nice.

One well-known and correct way to combine the coarse grid correction with one of the previous smoothers is to combine them additively,  to get an **additive two-grid preconditioner** as follows.

In [ ]:
twogrid = coarsepre + blockgs 

This addition of two operators (of type `BaseMatrix`) results in another operator, which is stored as an expression, to be evaluated only when needed.  The 2-grid preconditioner has a very good condition number.

In [ ]:
EigenValues_Preconditioner(mat=a.mat, pre=twogrid)